# Extract the quadrat from all 732 videos in the dataset

> Develop Python scripts to organize and pre-process underwater mussel image data for ingestion by computer vision model, including:
>> a. Extraction of stills from video sequence

### To successfully extract a quadrat the algorithm assumes:

- See the list of assumptions in notebook `task_1b_extract-quadrat-from-stills.ipynb`, or in the final report section titled __Quadrat Extraction__.

### Output

This notebook is configured for efficiency and has *no interactive output*. On completion there will be a folder for every video created in `SAVE_PATH` and the extracted quadrat contents (if any) as a jpeg file will be saved in each folder with the frame number as suffix.

In [1]:
import os
import os.path as osp
import sys

# Check if notebook is running in Colab or local workstation
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Search for all video files on Google Drive...
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = r'/content/drive/My Drive/Data'
    
    # cd into current directory so local imports work
    %cd '/content/drive/My Drive/cciw-zebra-mussel/quadrat-extraction/'
    
    # clone repo, install packages
else:
    DATA_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/Data')
    SAVE_PATH = osp.join(os.environ['DATA_PATH'], 'cciw/dataset_raw/quadrat-extraction/videos')

In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from utils import crop_still_image
from utils import compute_pairwise_distances

In [3]:
cv2.__version__

'4.2.0'

In [4]:
DRAW = False

skipFrames = 10

# indices of data in x and y position respectively
X, Y = 0, 1

In [11]:
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/2014/*/Videos/Quad*/*.mp4'))
videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)
print('Found %d videos' % len(all_videos))

Found 289 videos


In [ ]:
for video_index, file in enumerate(all_videos):
    
    video_name = file.split('/')[-1].split('.')[0]
    
    output_folder = os.path.join(SAVE_PATH, video_name)
    if not osp.exists(output_folder):
        os.makedirs(output_folder)
        print('Made output folder ', output_folder)
    else:
        print('Output folder %s already exists' % output_folder)
        continue

    cap = cv2.VideoCapture(file)
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
          int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print('Raw input resolution', sz)

    # read first frame to adjust resolution of output stream
    ret, im = cap.read() 

    # set additional meta-parameters according to input res
    if sz[0] == 1440:
        """x_trim and y_trim are used to remove black padding 
        which triggers spurious edges"""
        x_trim, y_trim = 1, 145
        im = im[y_trim:-y_trim, x_trim:-x_trim, :]
        crop_frame_border = True

        """@param canny_thresh# hysteresis values for Canny edge 
        detector, input to HoughLines"""
        canny_thresh1, canny_thresh2 = 10, 45

        """@param threshold Accumulator threshold, return 
        lines with more than threshold of votes. (intersection points)"""
        threshold = 125

        """@param minLineLength Minimum line length. 
        Line segments shorter than that are rejected. (pixels)"""
        mLL = 400

        """@param maxLineGap Maximum allowed gap between points 
        on the same line to link them. (pixels)"""
        mLG = 150
    else:
        # params as described above
        canny_thresh1, canny_thresh2 = 40, 100
        threshold = 125
        mLG, mLL = 150, 500
        crop_frame_border = False

    """this method may downsample, so set the video writer 
    resolution to the processed image resolution"""
    img, _, crop = crop_still_image(
            im, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2, do_draw=DRAW)
    sz = (img.shape[1], img.shape[0])
    print(sz)

    if cap.isOpened():
        print('Opened stream for writing, output resolution is', sz)
    else:
        print('cap is not open')


    currentFrame = 0
    """it can take 30s-1min to process entire video, 
    can optionally process a small number of frames"""

    # to process whole video    
    while(True):

        # Capture frame-by-frame
        for i in range(skipFrames):
            ret, im = cap.read()
        if not ret: break

        if crop_frame_border:
            im = im[y_trim:-y_trim, x_trim:-x_trim, :]

        # Do processing
        img, _, crop = crop_still_image(
            im, mll=mLL, mlg=mLG, threshold=threshold, canny_1=canny_thresh1, canny_2=canny_thresh2, do_draw=DRAW)

        # save still image in jpeg format
        try:
            x_start = crop[:, X].min()
            x_end = crop[:, X].max()
            y_start = crop[:, Y].min()
            y_end = crop[:, Y].max()

            if (compute_pairwise_distances(crop)[:, 2] < mLL).sum():
                print('Cannot crop: found corners do not form a square')
            else:
                try:
                    out_file = osp.join(
                        output_folder, video_name + '_crop_frame-%d.jpg' % currentFrame)
                    cv2.imwrite(out_file, img[y_start:y_end, x_start:x_end, :])
                except:
                    print('Cannot write ', out_file)
        except:
            pass

        # increment frame counter
        currentFrame += skipFrames
        
        if currentFrame > 800:
            break
    cap.release()

Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-2_2014-08-19_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-3_2014-08-19_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-1_2014-08-19_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-2_2014-09-24_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-3_2014-09-24_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-1_2014-09-24_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-2_2014-05-28_video-1 already exists
Output folder /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1353-3_2014-05-28_video-1 already exists
Output folder /scratch/gallowaa/

Using 9 of 14 lines
Cannot crop: found corners do not form a square
Using 10 of 12 lines
Cannot crop: found corners do not form a square
Using 9 of 13 lines
Cannot crop: found corners do not form a square
Using 10 of 12 lines
Cannot crop: found corners do not form a square
Using 10 of 14 lines
Cannot crop: found corners do not form a square
Using 6 of 14 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 6 of 9 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 9 of 14 lines
Cannot crop: found corners do not form a square
Using 8 of 10 lines
Cannot crop: found corners do not form a square
Using 8 of 12 lines
Cannot crop: found corners do not form a square
Using 10 of 14 lines
Using 12 of 14 lines
Made output folder  /scratch/gallowaa/cciw/dataset_raw/qua

Using 5 of 6 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 6 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 7 of 12 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 5 of 12 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 5 of 12 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 4 of 8 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 6 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visib

Cannot crop: found corners do not form a square
Using 3 of 6 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 2 of 3 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 4 of 6 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 2 of 10 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 1 of 3 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 4 of 11 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 1 of 5 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 3 of 9 lines
It is likely that: 
 i)  all four q

Cannot crop: found corners do not form a square
Made output folder  /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1347-1_2014-07-14_video-1
Raw input resolution (1920, 1080)
Using 8 of 14 lines
(1916, 1076)
Opened stream for writing, output resolution is (1916, 1076)
Using 7 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 10 of 13 lines
Cannot crop: found corners do not form a square
Using 1 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 6 of 12 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 3 of 14 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 3 of 11 lines
It is likel

...

Cannot crop: found corners do not form a square
Using 10 of 11 lines
Cannot crop: found corners do not form a square
Using 8 of 13 lines
Cannot crop: found corners do not form a square
Using 9 of 12 lines
Using 3 of 4 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Made output folder  /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1352-2_2014-09-26_video-1
Raw input resolution (1920, 1080)
Using 9 of 13 lines
(1916, 1076)
Opened stream for writing, output resolution is (1916, 1076)
Using 9 of 13 lines
Cannot crop: found corners do not form a square
Using 8 of 10 lines
Using 7 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 6 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 2 of 5 lines
It is likely that: 
 i)  all four quadtrat corners are not vis

Using 9 of 12 lines
Cannot crop: found corners do not form a square
Using 6 of 11 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 8 of 10 lines
Using 9 of 14 lines
Using 10 of 14 lines
Using 9 of 11 lines
Cannot crop: found corners do not form a square
Using 11 of 14 lines
Cannot crop: found corners do not form a square
Made output folder  /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1345-2_2014-05-29_video-1
Raw input resolution (1920, 1080)
Using 8 of 14 lines
(1916, 1076)
Opened stream for writing, output resolution is (1916, 1076)
Using 10 of 13 lines
Using 11 of 14 lines
Using 10 of 13 lines
Cannot crop: found corners do not form a square
Using 10 of 14 lines
Cannot crop: found corners do not form a square
Using 11 of 14 lines
Cannot crop: found corners do not form a square
Using 7 of 10 lines
It is likely that: 
 i)  all four quadtrat corne

Cannot crop: found corners do not form a square
Using 5 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 8 of 12 lines
Cannot crop: found corners do not form a square
Using 8 of 14 lines
Cannot crop: found corners do not form a square
Using 7 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 6 of 11 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Cannot crop: found corners do not form a square
Using 4 of 12 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 4 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 6 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are 

Using 9 of 14 lines
Using 10 of 13 lines
Using 10 of 14 lines
Using 7 of 11 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 8 of 13 lines
Using 6 of 8 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 8 of 14 lines
Using 6 of 11 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 5 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 6 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 8 of 10 lines
Using 6 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 10 of 14 lines
Cannot crop: found corners do not form a square
Using 9 of 14 lines
Cannot crop: fou

Using 10 of 14 lines
Cannot crop: found corners do not form a square
Using 11 of 14 lines
Cannot crop: found corners do not form a square
Using 8 of 12 lines
Using 8 of 12 lines
Using 5 of 10 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Made output folder  /scratch/gallowaa/cciw/dataset_raw/quadrat-extraction/videos/GLNI_1345-1_2014-09-25_video-1
Raw input resolution (1920, 1080)
Using 3 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
(1916, 1076)
Opened stream for writing, output resolution is (1916, 1076)
Using 1 of 14 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 1 of 13 lines
It is likely that: 
 i)  all four quadtrat corners are not visible 
 ii) image is blurry, or partly occluded
Using 3 of 11 lines
It is likely that: 
 i)  all four quadtrat corners are not visible